In [ ]:
#Import libraries
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import pytz
from datetime import datetime
from datetime import timedelta

In [ ]:
# display data on the MetaTrader 5 package
print("MetaTrader5 package author: ",mt5.__author__)
print("MetaTrader5 package version: ",mt5.__version__)


In [ ]:
import MetaTrader5 as mt5

# Initialize the MetaTrader 5 platform
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

login = 48438229  # Your account number
password = "Mq3p8!tJ"  # Your account password
server = "HFMarketsGlobal-Demo"  # Your broker's server

# Login to the trading account
if not mt5.login(login, password, server):
    print(f"Failed to connect to account {login}")
    mt5.shutdown()

# Ensure the symbol is available
symbol = "GBPUSD"
symbol_info = mt5.symbol_info(symbol)
if symbol_info is None:
    print(f"Symbol {symbol} not found, cannot call order_check()")
elif not symbol_info.visible:
    print(f"Symbol {symbol} is not visible, trying to switch on")
    if not mt5.symbol_select(symbol, True):
        print(f"Symbol {symbol} is not available for trading")

# Get current price
symbol_tick = mt5.symbol_info_tick(symbol)
ask_price = symbol_tick.ask
bid_price = symbol_tick.bid

# Define the lot size
lot = 0.1

# Calculate stop loss and take profit prices for buy stop and sell stop orders
pip = 0.0001  # 1 pip for EURUSD is 0.0001
stop_loss_pips = 50
take_profit_pips = 50
stop_pips = 10

stop_loss_price_buy = ask_price - stop_loss_pips * pip
take_profit_price_buy = ask_price + take_profit_pips * pip
stop_price_buy = ask_price + stop_pips * pip

stop_loss_price_sell = bid_price + stop_loss_pips * pip
take_profit_price_sell = bid_price - take_profit_pips * pip
stop_price_sell = bid_price - stop_pips * pip

# Prepare the buy stop request
buy_stop_request = {
    "action": mt5.TRADE_ACTION_PENDING,
    "symbol": symbol,
    "volume": lot,
    "type": mt5.ORDER_TYPE_BUY_STOP,
    "price": stop_price_buy,
    "sl": stop_loss_price_buy,
    "tp": take_profit_price_buy,
    "deviation": 10,
    "magic": 234000,
    "comment": "python script buy stop",
    "type_time": mt5.ORDER_TIME_GTC,
    "type_filling": mt5.ORDER_FILLING_FOK,  # Using FOK filling mode
}

# Prepare the sell stop request
sell_stop_request = {
    "action": mt5.TRADE_ACTION_PENDING,
    "symbol": symbol,
    "volume": lot,
    "type": mt5.ORDER_TYPE_SELL_STOP,
    "price": stop_price_sell,
    "sl": stop_loss_price_sell,
    "tp": take_profit_price_sell,
    "deviation": 10,
    "magic": 234000,
    "comment": "python script sell stop",
    "type_time": mt5.ORDER_TIME_GTC,
    "type_filling": mt5.ORDER_FILLING_FOK,  # Using FOK filling mode
}

# Send the buy stop request
result_buy_stop = mt5.order_send(buy_stop_request)

# Check the buy stop result
if result_buy_stop.retcode != mt5.TRADE_RETCODE_DONE:
    print(f"Buy stop order failed, retcode={result_buy_stop.retcode}")
    print(f"Error: {result_buy_stop}")
else:
    print(f"Buy stop order placed successfully, ticket={result_buy_stop.order}")

# Send the sell stop request
result_sell_stop = mt5.order_send(sell_stop_request)

# Check the sell stop result
if result_sell_stop.retcode != mt5.TRADE_RETCODE_DONE:
    print(f"Sell stop order failed, retcode={result_sell_stop.retcode}")
    print(f"Error: {result_sell_stop}")
else:
    print(f"Sell stop order placed successfully, ticket={result_sell_stop.order}")

# Shutdown the MetaTrader 5 platform
mt5.shutdown()


# Adding trailing Stop

In [ ]:
import MetaTrader5 as mt5
import time

# Initialize the MetaTrader 5 platform
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()
    exit()

login = 48438229  # Your account number
password = "Mq3p8!tJ"  # Your account password
server = "HFMarketsGlobal-Demo"  # Your broker's server

# Login to the trading account
if not mt5.login(login, password, server):
    print(f"Failed to connect to account {login}")
    mt5.shutdown()
    exit()

# Ensure AutoTrading is enabled
account_info = mt5.account_info()
if account_info is None:
    print("Failed to get account info, AutoTrading might be disabled")
    mt5.shutdown()
    exit()
elif not account_info.trade_allowed:
    print("AutoTrading is disabled, please enable it in the MetaTrader 5 platform")
    mt5.shutdown()
    exit()

# Ensure the symbol is available
symbol = "GBPUSD"
symbol_info = mt5.symbol_info(symbol)
if symbol_info is None:
    print(f"Symbol {symbol} not found, cannot call order_check()")
    mt5.shutdown()
    exit()
elif not symbol_info.visible:
    print(f"Symbol {symbol} is not visible, trying to switch on")
    if not mt5.symbol_select(symbol, True):
        print(f"Symbol {symbol} is not available for trading")
        mt5.shutdown()
        exit()

# Get current price
symbol_tick = mt5.symbol_info_tick(symbol)
ask_price = symbol_tick.ask
bid_price = symbol_tick.bid

# Define the lot size
lot = 1.0

# Calculate stop loss and stop prices for buy stop and sell stop orders
pip = 0.0001  # 1 pip for EURUSD is 0.0001
stop_loss_pips = 50
trigger_pips = 40
trailing_pips = 20
stop_pips = 3

stop_loss_price_buy = ask_price - stop_loss_pips * pip
stop_price_buy = ask_price + stop_pips * pip

stop_loss_price_sell = bid_price + stop_loss_pips * pip
stop_price_sell = bid_price - stop_pips * pip

# Prepare the buy stop request
buy_stop_request = {
    "action": mt5.TRADE_ACTION_PENDING,
    "symbol": symbol,
    "volume": lot,
    "type": mt5.ORDER_TYPE_BUY_STOP,
    "price": stop_price_buy,
    "sl": stop_loss_price_buy,
    "deviation": 10,
    "magic": 234000,
    "comment": "python script buy stop",
    "type_time": mt5.ORDER_TIME_GTC,
    "type_filling": mt5.ORDER_FILLING_FOK,  # Using FOK filling mode
}

# Prepare the sell stop request
sell_stop_request = {
    "action": mt5.TRADE_ACTION_PENDING,
    "symbol": symbol,
    "volume": lot,
    "type": mt5.ORDER_TYPE_SELL_STOP,
    "price": stop_price_sell,
    "sl": stop_loss_price_sell,
    "deviation": 10,
    "magic": 234000,
    "comment": "python script sell stop",
    "type_time": mt5.ORDER_TIME_GTC,
    "type_filling": mt5.ORDER_FILLING_FOK,  # Using FOK filling mode
}

# Send the buy stop request
result_buy_stop = mt5.order_send(buy_stop_request)

# Check the buy stop result
if result_buy_stop.retcode != mt5.TRADE_RETCODE_DONE:
    print(f"Buy stop order failed, retcode={result_buy_stop.retcode}")
    print(f"Error: {result_buy_stop}")
else:
    print(f"Buy stop order placed successfully, ticket={result_buy_stop.order}")

# Send the sell stop request
result_sell_stop = mt5.order_send(sell_stop_request)

# Check the sell stop result
if result_sell_stop.retcode != mt5.TRADE_RETCODE_DONE:
    print(f"Sell stop order failed, retcode={result_sell_stop.retcode}")
    print(f"Error: {result_sell_stop}")
else:
    print(f"Sell stop order placed successfully, ticket={result_sell_stop.order}")

# Function to update trailing stop loss
def update_trailing_stop(position, trailing_stop):
    current_price = mt5.symbol_info_tick(position.symbol).bid if position.type == mt5.ORDER_TYPE_BUY else mt5.symbol_info_tick(position.symbol).ask
    new_stop_loss = current_price - trailing_stop * pip if position.type == mt5.ORDER_TYPE_BUY else current_price + trailing_stop * pip
    request = {
        "action": mt5.TRADE_ACTION_SLTP,
        "symbol": position.symbol,
        "volume": position.volume,
        "position": position.ticket,
        "sl": new_stop_loss,
        "tp": position.tp,
        "deviation": 10,
        "magic": position.magic,
        "comment": "python script trailing stop",
    }
    result = mt5.order_send(request)
    if result.retcode != mt5.TRADE_RETCODE_DONE:
        print(f"Failed to modify stop loss for position {position.ticket}, retcode={result.retcode}")
    else:
        print(f"Trailing stop updated for position {position.ticket}")

# Monitor positions and update trailing stop if necessary
while True:
    positions = mt5.positions_get(symbol=symbol)
    for position in positions:
        if position.type == mt5.ORDER_TYPE_BUY:
            current_price = mt5.symbol_info_tick(symbol).bid
            if current_price - position.price_open >= trigger_pips * pip:
                update_trailing_stop(position, trailing_pips)
        elif position.type == mt5.ORDER_TYPE_SELL:
            current_price = mt5.symbol_info_tick(symbol).ask
            if position.price_open - current_price >= trigger_pips * pip:
                update_trailing_stop(position, trailing_pips)
    time.sleep(10)  # Wait for 10 seconds before checking again

# Shutdown the MetaTrader 5 platform
mt5.shutdown()


In [ ]:
# Adding Timer and it works

In [ ]:
import MetaTrader5 as mt5
import time
from datetime import datetime, timedelta

# Initialize the MetaTrader 5 platform
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()
    exit()

login = 48438229  # Your account number
password = "Mq3p8!tJ"  # Your account password
server = "HFMarketsGlobal-Demo"  # Your broker's server

# Login to the trading account
if not mt5.login(login, password, server):
    print(f"Failed to connect to account {login}")
    mt5.shutdown()
    exit()

# Ensure AutoTrading is enabled
account_info = mt5.account_info()
if account_info is None:
    print("Failed to get account info, AutoTrading might be disabled")
    mt5.shutdown()
    exit()
elif not account_info.trade_allowed:
    print("AutoTrading is disabled, please enable it in the MetaTrader 5 platform")
    mt5.shutdown()
    exit()

# Ensure the symbol is available
symbol = "GBPUSD"
symbol_info = mt5.symbol_info(symbol)
if symbol_info is None:
    print(f"Symbol {symbol} not found, cannot call order_check()")
    mt5.shutdown()
    exit()
elif not symbol_info.visible:
    print(f"Symbol {symbol} is not visible, trying to switch on")
    if not mt5.symbol_select(symbol, True):
        print(f"Symbol {symbol} is not available for trading")
        mt5.shutdown()
        exit()

# Set the specific date and time to place the orders
scheduled_time = datetime(2024, 6, 14, 16, 37, 0)  # Change to your desired date and time

# Define the lot size and pip parameters
lot = 1.0
pip = 0.0001  # 1 pip for EURUSD is 0.0001
stop_loss_pips = 10
trigger_pips = 15
trailing_pips = 13
stop_pips = 10

# Function to place the orders
def place_orders():
    symbol_tick = mt5.symbol_info_tick(symbol)
    ask_price = symbol_tick.ask
    bid_price = symbol_tick.bid

    stop_loss_price_buy = ask_price - stop_loss_pips * pip
    stop_price_buy = ask_price + stop_pips * pip

    stop_loss_price_sell = bid_price + stop_loss_pips * pip
    stop_price_sell = bid_price - stop_pips * pip

    buy_stop_request = {
        "action": mt5.TRADE_ACTION_PENDING,
        "symbol": symbol,
        "volume": lot,
        "type": mt5.ORDER_TYPE_BUY_STOP,
        "price": stop_price_buy,
        "sl": stop_loss_price_buy,
        "deviation": 10,
        "magic": 234000,
        "comment": "python script buy stop",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_FOK,  # Using FOK filling mode
    }

    sell_stop_request = {
        "action": mt5.TRADE_ACTION_PENDING,
        "symbol": symbol,
        "volume": lot,
        "type": mt5.ORDER_TYPE_SELL_STOP,
        "price": stop_price_sell,
        "sl": stop_loss_price_sell,
        "deviation": 10,
        "magic": 234000,
        "comment": "python script sell stop",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_FOK,  # Using FOK filling mode
    }

    result_buy_stop = mt5.order_send(buy_stop_request)
    if result_buy_stop.retcode != mt5.TRADE_RETCODE_DONE:
        print(f"Buy stop order failed, retcode={result_buy_stop.retcode}")
        print(f"Error: {result_buy_stop}")
    else:
        print(f"Buy stop order placed successfully, ticket={result_buy_stop.order}")

    result_sell_stop = mt5.order_send(sell_stop_request)
    if result_sell_stop.retcode != mt5.TRADE_RETCODE_DONE:
        print(f"Sell stop order failed, retcode={result_sell_stop.retcode}")
        print(f"Error: {result_sell_stop}")
    else:
        print(f"Sell stop order placed successfully, ticket={result_sell_stop.order}")

# Wait until the scheduled time to place the orders
while datetime.now() < scheduled_time:
    time.sleep(1)

# Place the orders
place_orders()

# Shutdown the MetaTrader 5 platform
mt5.shutdown()


# Show countdown on here, works too

In [4]:
import MetaTrader5 as mt5
import time
from datetime import datetime

# Initialize the MetaTrader 5 platform
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()
    exit()

login = 48438229  # Your account number
password = "Mq3p8!tJ"  # Your account password
server = "HFMarketsGlobal-Demo"  # Your broker's server

# Login to the trading account
if not mt5.login(login, password, server):
    print(f"Failed to connect to account {login}")
    mt5.shutdown()
    exit()

# Ensure AutoTrading is enabled
account_info = mt5.account_info()
if account_info is None:
    print("Failed to get account info, AutoTrading might be disabled")
    mt5.shutdown()
    exit()
elif not account_info.trade_allowed:
    print("AutoTrading is disabled, please enable it in the MetaTrader 5 platform")
    mt5.shutdown()
    exit()

# Ensure the symbol is available
symbol = "XAUUSD"
symbol_info = mt5.symbol_info(symbol)
if symbol_info is None:
    print(f"Symbol {symbol} not found, cannot call order_check()")
    mt5.shutdown()
    exit()
elif not symbol_info.visible:
    print(f"Symbol {symbol} is not visible, trying to switch on")
    if not mt5.symbol_select(symbol, True):
        print(f"Symbol {symbol} is not available for trading")
        mt5.shutdown()
        exit()

# Set the specific date and time to place the orders
scheduled_time = datetime(2024, 6, 18, 12, 55, 0)  # Change to your desired date and time

# Define the lot size and pip parameters
lot = 0.01
pip = 0.1  # 1 pip for EURUSD is 0.0001
stop_loss_pips = 10
trigger_pips = 20
trailing_pips = 15
stop_pips = 10

# Function to place the orders
def place_orders():
    symbol_tick = mt5.symbol_info_tick(symbol)
    ask_price = symbol_tick.ask
    bid_price = symbol_tick.bid

    stop_loss_price_buy = ask_price - stop_loss_pips * pip
    stop_price_buy = ask_price + stop_pips * pip

    stop_loss_price_sell = bid_price + stop_loss_pips * pip
    stop_price_sell = bid_price - stop_pips * pip

    buy_stop_request = {
        "action": mt5.TRADE_ACTION_PENDING,
        "symbol": symbol,
        "volume": lot,
        "type": mt5.ORDER_TYPE_BUY_STOP,
        "price": stop_price_buy,
        "sl": stop_loss_price_buy,
        "deviation": 10,
        "magic": 234000,
        "comment": "python script buy stop",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_FOK,  # Using FOK filling mode
    }

    sell_stop_request = {
        "action": mt5.TRADE_ACTION_PENDING,
        "symbol": symbol,
        "volume": lot,
        "type": mt5.ORDER_TYPE_SELL_STOP,
        "price": stop_price_sell,
        "sl": stop_loss_price_sell,
        "deviation": 10,
        "magic": 234000,
        "comment": "python script sell stop",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_FOK,  # Using FOK filling mode
    }

    result_buy_stop = mt5.order_send(buy_stop_request)
    if result_buy_stop.retcode != mt5.TRADE_RETCODE_DONE:
        print(f"Buy stop order failed, retcode={result_buy_stop.retcode}")
        print(f"Error: {result_buy_stop}")
    else:
        print(f"Buy stop order placed successfully, ticket={result_buy_stop.order}")

    result_sell_stop = mt5.order_send(sell_stop_request)
    if result_sell_stop.retcode != mt5.TRADE_RETCODE_DONE:
        print(f"Sell stop order failed, retcode={result_sell_stop.retcode}")
        print(f"Error: {result_sell_stop}")
    else:
        print(f"Sell stop order placed successfully, ticket={result_sell_stop.order}")

# Function to display the countdown timer
def display_countdown(target_time):
    while datetime.now() < target_time:
        now = datetime.now()
        remaining_time = target_time - now
        hours, remainder = divmod(remaining_time.seconds, 3600)
        minutes, seconds = divmod(remainder, 60)
        print(f"Time remaining: {remaining_time.days} days, {hours:02}:{minutes:02}:{seconds:02}", end="\r")
        time.sleep(1)

# Display the countdown timer
display_countdown(scheduled_time)

# Place the orders
place_orders()

# Shutdown the MetaTrader 5 platform
mt5.shutdown()


Buy stop order placed successfully, ticket=83732483
Sell stop order placed successfully, ticket=83732485


True

In [ ]:
# Closing Pending trades not taken in 3 mins

In [3]:
import MetaTrader5 as mt5
import time
from datetime import datetime, timedelta

# Initialize the MetaTrader 5 platform
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()
    exit()

login = 48438229  # Your account number
password = "Mq3p8!tJ"  # Your account password
server = "HFMarketsGlobal-Demo"  # Your broker's server

# Login to the trading account
if not mt5.login(login, password, server):
    print(f"Failed to connect to account {login}")
    mt5.shutdown()
    exit()

# Ensure AutoTrading is enabled
account_info = mt5.account_info()
if account_info is None:
    print("Failed to get account info, AutoTrading might be disabled")
    mt5.shutdown()
    exit()
elif not account_info.trade_allowed:
    print("AutoTrading is disabled, please enable it in the MetaTrader 5 platform")
    mt5.shutdown()
    exit()

# Ensure the symbol is available
symbol = "AUDUSD"
symbol_info = mt5.symbol_info(symbol)
if symbol_info is None:
    print(f"Symbol {symbol} not found, cannot call order_check()")
    mt5.shutdown()
    exit()
elif not symbol_info.visible:
    print(f"Symbol {symbol} is not visible, trying to switch on")
    if not mt5.symbol_select(symbol, True):
        print(f"Symbol {symbol} is not available for trading")
        mt5.shutdown()
        exit()

# Set the specific date and time to place the orders
scheduled_time = datetime(2024, 6, 14, 17, 32, 0)  # Change to your desired date and time

# Define the lot size and pip parameters
lot = 1.0
pip = 0.0001  # 1 pip for EURUSD is 0.0001
stop_loss_pips = 10
trigger_pips = 17
trailing_pips = 13
stop_pips = 10

# Function to place the orders
def place_orders():
    symbol_tick = mt5.symbol_info_tick(symbol)
    ask_price = symbol_tick.ask
    bid_price = symbol_tick.bid

    stop_loss_price_buy = ask_price - stop_loss_pips * pip
    stop_price_buy = ask_price + stop_pips * pip

    stop_loss_price_sell = bid_price + stop_loss_pips * pip
    stop_price_sell = bid_price - stop_pips * pip

    buy_stop_request = {
        "action": mt5.TRADE_ACTION_PENDING,
        "symbol": symbol,
        "volume": lot,
        "type": mt5.ORDER_TYPE_BUY_STOP,
        "price": stop_price_buy,
        "sl": stop_loss_price_buy,
        "deviation": 10,
        "magic": 234000,
        "comment": "python script buy stop",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_FOK,  # Using FOK filling mode
    }

    sell_stop_request = {
        "action": mt5.TRADE_ACTION_PENDING,
        "symbol": symbol,
        "volume": lot,
        "type": mt5.ORDER_TYPE_SELL_STOP,
        "price": stop_price_sell,
        "sl": stop_loss_price_sell,
        "deviation": 10,
        "magic": 234000,
        "comment": "python script sell stop",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_FOK,  # Using FOK filling mode
    }

    result_buy_stop = mt5.order_send(buy_stop_request)
    if result_buy_stop.retcode != mt5.TRADE_RETCODE_DONE:
        print(f"Buy stop order failed, retcode={result_buy_stop.retcode}")
        print(f"Error: {result_buy_stop}")
    else:
        print(f"Buy stop order placed successfully, ticket={result_buy_stop.order}")

    result_sell_stop = mt5.order_send(sell_stop_request)
    if result_sell_stop.retcode != mt5.TRADE_RETCODE_DONE:
        print(f"Sell stop order failed, retcode={result_sell_stop.retcode}")
        print(f"Error: {result_sell_stop}")
    else:
        print(f"Sell stop order placed successfully, ticket={result_sell_stop.order}")

# Function to close pending orders not executed within 3 minutes
def close_pending_orders():
    three_minutes_after = scheduled_time + timedelta(minutes=3)
    while datetime.now() < three_minutes_after:
        time.sleep(1)

    orders = mt5.orders_get(symbol=symbol)
    if orders is None:
        print("No orders to close.")
        return

    for order in orders:
        if order.type in [mt5.ORDER_TYPE_BUY_STOP, mt5.ORDER_TYPE_SELL_STOP]:
            close_request = {
                "action": mt5.TRADE_ACTION_REMOVE,
                "order": order.ticket,
                "symbol": order.symbol,
                "volume": order.volume_current,
                "magic": order.magic,
                "comment": "python script cancel pending",
            }
            result = mt5.order_send(close_request)
            if result.retcode != mt5.TRADE_RETCODE_DONE:
                print(f"Failed to close order {order.ticket}, retcode={result.retcode}")
            else:
                print(f"Order {order.ticket} closed successfully.")

# Function to display the countdown timer
def display_countdown(target_time):
    while datetime.now() < target_time:
        now = datetime.now()
        remaining_time = target_time - now
        hours, remainder = divmod(remaining_time.seconds, 3600)
        minutes, seconds = divmod(remainder, 60)
        print(f"Time remaining: {remaining_time.days} days, {hours:02}:{minutes:02}:{seconds:02}", end="\r")
        time.sleep(1)

# Display the countdown timer
display_countdown(scheduled_time)

# Place the orders
place_orders()

# Close pending orders not taken after 3 minutes
close_pending_orders()

# Shutdown the MetaTrader 5 platform
mt5.shutdown()


Buy stop order placed successfully, ticket=83351587
Sell stop order placed successfully, ticket=83351588
Order 83351587 closed successfully.
Order 83351588 closed successfully.


True

# 3 different times and it works too

In [4]:
import MetaTrader5 as mt5
import time
from datetime import datetime, timedelta

# Initialize the MetaTrader 5 platform
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()
    exit()

login = 48438229  # Your account number
password = "Mq3p8!tJ"  # Your account password
server = "HFMarketsGlobal-Demo"  # Your broker's server

# Login to the trading account
if not mt5.login(login, password, server):
    print(f"Failed to connect to account {login}")
    mt5.shutdown()
    exit()

# Ensure AutoTrading is enabled
account_info = mt5.account_info()
if account_info is None:
    print("Failed to get account info, AutoTrading might be disabled")
    mt5.shutdown()
    exit()
elif not account_info.trade_allowed:
    print("AutoTrading is disabled, please enable it in the MetaTrader 5 platform")
    mt5.shutdown()
    exit()

# Ensure the symbol is available
symbol = "EURUSD"
symbol_info = mt5.symbol_info(symbol)
if symbol_info is None:
    print(f"Symbol {symbol} not found, cannot call order_check()")
    mt5.shutdown()
    exit()
elif not symbol_info.visible:
    print(f"Symbol {symbol} is not visible, trying to switch on")
    if not mt5.symbol_select(symbol, True):
        print(f"Symbol {symbol} is not available for trading")
        mt5.shutdown()
        exit()

# Set the specific dates and times to place the orders
scheduled_times = [
    datetime(2024, 6, 18, 13, 28, 0),  # First timer
    datetime(2024, 6, 18, 13, 39, 0),  # Second timer
    datetime(2024, 6, 18, 13, 45, 0)   # Third timer
]

# Define the lot size and pip parameters
lot = 1.0
pip = 0.0001  # 1 pip for EURUSD is 0.0001
stop_loss_pips = 10
trigger_pips = 15
trailing_pips = 15
stop_pips = 10

# Function to place the orders
def place_orders():
    symbol_tick = mt5.symbol_info_tick(symbol)
    ask_price = symbol_tick.ask
    bid_price = symbol_tick.bid

    stop_loss_price_buy = ask_price - stop_loss_pips * pip
    stop_price_buy = ask_price + stop_pips * pip

    stop_loss_price_sell = bid_price + stop_loss_pips * pip
    stop_price_sell = bid_price - stop_pips * pip

    buy_stop_request = {
        "action": mt5.TRADE_ACTION_PENDING,
        "symbol": symbol,
        "volume": lot,
        "type": mt5.ORDER_TYPE_BUY_STOP,
        "price": stop_price_buy,
        "sl": stop_loss_price_buy,
        "deviation": 10,
        "magic": 234000,
        "comment": "python script buy stop",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_FOK,  # Using FOK filling mode
    }

    sell_stop_request = {
        "action": mt5.TRADE_ACTION_PENDING,
        "symbol": symbol,
        "volume": lot,
        "type": mt5.ORDER_TYPE_SELL_STOP,
        "price": stop_price_sell,
        "sl": stop_loss_price_sell,
        "deviation": 10,
        "magic": 234000,
        "comment": "python script sell stop",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_FOK,  # Using FOK filling mode
    }

    result_buy_stop = mt5.order_send(buy_stop_request)
    if result_buy_stop.retcode != mt5.TRADE_RETCODE_DONE:
        print(f"Buy stop order failed, retcode={result_buy_stop.retcode}")
        print(f"Error: {result_buy_stop}")
    else:
        print(f"Buy stop order placed successfully, ticket={result_buy_stop.order}")

    result_sell_stop = mt5.order_send(sell_stop_request)
    if result_sell_stop.retcode != mt5.TRADE_RETCODE_DONE:
        print(f"Sell stop order failed, retcode={result_sell_stop.retcode}")
        print(f"Error: {result_sell_stop}")
    else:
        print(f"Sell stop order placed successfully, ticket={result_sell_stop.order}")

# Function to close pending orders not executed within 3 minutes
def close_pending_orders(after_time):
    close_time = after_time + timedelta(minutes=3)
    while datetime.now() < close_time:
        time.sleep(1)

    orders = mt5.orders_get(symbol=symbol)
    if orders is None:
        print("No orders to close.")
        return

    for order in orders:
        if order.type in [mt5.ORDER_TYPE_BUY_STOP, mt5.ORDER_TYPE_SELL_STOP]:
            close_request = {
                "action": mt5.TRADE_ACTION_REMOVE,
                "order": order.ticket,
                "symbol": order.symbol,
                "volume": order.volume_current,
                "magic": order.magic,
                "comment": "python script cancel pending",
            }
            result = mt5.order_send(close_request)
            if result.retcode != mt5.TRADE_RETCODE_DONE:
                print(f"Failed to close order {order.ticket}, retcode={result.retcode}")
            else:
                print(f"Order {order.ticket} closed successfully.")

# Function to display the countdown timer
def display_countdown(target_time):
    while datetime.now() < target_time:
        now = datetime.now()
        remaining_time = target_time - now
        hours, remainder = divmod(remaining_time.seconds, 3600)
        minutes, seconds = divmod(remainder, 60)
        print(f"Time remaining: {remaining_time.days} days, {hours:02}:{minutes:02}:{seconds:02}", end="\r")
        time.sleep(1)

# Function to manage multiple timers
def manage_timers(scheduled_times):
    for scheduled_time in scheduled_times:
        # Display the countdown timer
        display_countdown(scheduled_time)

        # Place the orders
        place_orders()

        # Close pending orders not taken after 3 minutes
        close_pending_orders(scheduled_time)

# Manage multiple timers
manage_timers(scheduled_times)

# Shutdown the MetaTrader 5 platform
mt5.shutdown()





Buy stop order placed successfully, ticket=83359555
Sell stop order placed successfully, ticket=83359556
Order 83359555 closed successfully.
Order 83359556 closed successfully.
Buy stop order placed successfully, ticket=83360000
Sell stop order placed successfully, ticket=83360001
Order 83360000 closed successfully.
Order 83360001 closed successfully.
Buy stop order placed successfully, ticket=83360468
Sell stop order placed successfully, ticket=83360469
Order 83360468 closed successfully.
Order 83360469 closed successfully.


True

In [ ]:
# Now Adding multiple symbols or pairs but trailing stop doesnt work

In [5]:
import MetaTrader5 as mt5
import time
from datetime import datetime, timedelta

# Initialize the MetaTrader 5 platform
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()
    exit()

login = 48438229  # Your account number
password = "Mq3p8!tJ"  # Your account password
server = "HFMarketsGlobal-Demo"  # Your broker's server

# Login to the trading account
if not mt5.login(login, password, server):
    print(f"Failed to connect to account {login}")
    mt5.shutdown()
    exit()

# Ensure AutoTrading is enabled
account_info = mt5.account_info()
if account_info is None:
    print("Failed to get account info, AutoTrading might be disabled")
    mt5.shutdown()
    exit()
elif not account_info.trade_allowed:
    print("AutoTrading is disabled, please enable it in the MetaTrader 5 platform")
    mt5.shutdown()
    exit()

# List of symbols
symbols = ["EURUSD", "GBPUSD", "USDJPY", "XAUUSD"]

# Set the specific dates and times to place the orders
scheduled_times = [
    datetime(2024, 6, 18, 13, 28, 0),  # First timer
    datetime(2024, 6, 18, 13, 39, 0),  # Second timer
    datetime(2024, 6, 18, 13, 45, 0)   # Third timer
]

# Define the lot size and pip parameters
lot = 1.0
pip_dict = {"EURUSD": 0.0001, "GBPUSD": 0.0001, "USDJPY": 0.01, "XAUUSD": 0.1}  # Different pip values for different symbols
stop_loss_pips = 10
trigger_pips = 20
trailing_pips = 15
stop_pips = 10

# Function to ensure the symbol is available
def ensure_symbol(symbol):
    symbol_info = mt5.symbol_info(symbol)
    if symbol_info is None:
        print(f"Symbol {symbol} not found, cannot call order_check()")
        return False
    elif not symbol_info.visible:
        print(f"Symbol {symbol} is not visible, trying to switch on")
        if not mt5.symbol_select(symbol, True):
            print(f"Symbol {symbol} is not available for trading")
            return False
    return True

# Function to place the orders
def place_orders(symbol):
    if not ensure_symbol(symbol):
        return

    pip = pip_dict[symbol]
    symbol_tick = mt5.symbol_info_tick(symbol)
    ask_price = symbol_tick.ask
    bid_price = symbol_tick.bid

    stop_loss_price_buy = ask_price - stop_loss_pips * pip
    stop_price_buy = ask_price + stop_pips * pip

    stop_loss_price_sell = bid_price + stop_loss_pips * pip
    stop_price_sell = bid_price - stop_pips * pip

    buy_stop_request = {
        "action": mt5.TRADE_ACTION_PENDING,
        "symbol": symbol,
        "volume": lot,
        "type": mt5.ORDER_TYPE_BUY_STOP,
        "price": stop_price_buy,
        "sl": stop_loss_price_buy,
        "deviation": 10,
        "magic": 234000,
        "comment": "python script buy stop",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_FOK,  # Using FOK filling mode
    }

    sell_stop_request = {
        "action": mt5.TRADE_ACTION_PENDING,
        "symbol": symbol,
        "volume": lot,
        "type": mt5.ORDER_TYPE_SELL_STOP,
        "price": stop_price_sell,
        "sl": stop_loss_price_sell,
        "deviation": 10,
        "magic": 234000,
        "comment": "python script sell stop",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_FOK,  # Using FOK filling mode
    }

    result_buy_stop = mt5.order_send(buy_stop_request)
    if result_buy_stop.retcode != mt5.TRADE_RETCODE_DONE:
        print(f"Buy stop order for {symbol} failed, retcode={result_buy_stop.retcode}")
        print(f"Error: {result_buy_stop}")
    else:
        print(f"Buy stop order for {symbol} placed successfully, ticket={result_buy_stop.order}")

    result_sell_stop = mt5.order_send(sell_stop_request)
    if result_sell_stop.retcode != mt5.TRADE_RETCODE_DONE:
        print(f"Sell stop order for {symbol} failed, retcode={result_sell_stop.retcode}")
        print(f"Error: {result_sell_stop}")
    else:
        print(f"Sell stop order for {symbol} placed successfully, ticket={result_sell_stop.order}")

# Function to close pending orders not executed within 3 minutes
def close_pending_orders(symbol, after_time):
    close_time = after_time + timedelta(minutes=30)
    while datetime.now() < close_time:
        time.sleep(1)

    orders = mt5.orders_get(symbol=symbol)
    if orders is None:
        print(f"No orders to close for {symbol}.")
        return

    for order in orders:
        if order.type in [mt5.ORDER_TYPE_BUY_STOP, mt5.ORDER_TYPE_SELL_STOP]:
            close_request = {
                "action": mt5.TRADE_ACTION_REMOVE,
                "order": order.ticket,
                "symbol": order.symbol,
                "volume": order.volume_current,
                "magic": order.magic,
                "comment": "python script cancel pending",
            }
            result = mt5.order_send(close_request)
            if result.retcode != mt5.TRADE_RETCODE_DONE:
                print(f"Failed to close order {order.ticket} for {symbol}, retcode={result.retcode}")
            else:
                print(f"Order {order.ticket} for {symbol} closed successfully.")

# Function to display the countdown timer
def display_countdown(target_time):
    while datetime.now() < target_time:
        now = datetime.now()
        remaining_time = target_time - now
        hours, remainder = divmod(remaining_time.seconds, 3600)
        minutes, seconds = divmod(remainder, 60)
        print(f"Time remaining: {remaining_time.days} days, {hours:02}:{minutes:02}:{seconds:02}", end="\r")
        time.sleep(1)

# Function to manage multiple timers for multiple symbols
def manage_timers(symbols, scheduled_times):
    for scheduled_time in scheduled_times:
        
        # Display the countdown timer
        display_countdown(scheduled_time)

        # Place the orders for each symbol
        for symbol in symbols:
            place_orders(symbol)

        # Close pending orders not taken after 3 minutes for each symbol
        for symbol in symbols:
            close_pending_orders(symbol, scheduled_time)

# Manage multiple timers for multiple symbols
manage_timers(symbols, scheduled_times)

# Shutdown the MetaTrader 5 platform
mt5.shutdown()


Buy stop order for EURUSD placed successfully, ticket=83743195
Sell stop order for EURUSD placed successfully, ticket=83743196
Buy stop order for GBPUSD placed successfully, ticket=83743197
Sell stop order for GBPUSD placed successfully, ticket=83743198
Buy stop order for USDJPY placed successfully, ticket=83743200
Sell stop order for USDJPY placed successfully, ticket=83743201
Buy stop order for XAUUSD placed successfully, ticket=83743202
Sell stop order for XAUUSD placed successfully, ticket=83743204
Buy stop order for EURUSD placed successfully, ticket=83780961
Sell stop order for EURUSD placed successfully, ticket=83780965
Buy stop order for GBPUSD placed successfully, ticket=83780968
Sell stop order for GBPUSD placed successfully, ticket=83780972
Buy stop order for USDJPY placed successfully, ticket=83780974
Sell stop order for USDJPY placed successfully, ticket=83780980
Buy stop order for XAUUSD placed successfully, ticket=83780982
Sell stop order for XAUUSD placed successfully, 

True

In [ ]:
# Adjusted trailing stop so it would work

In [1]:
import MetaTrader5 as mt5
import time
from datetime import datetime, timedelta

# Initialize the MetaTrader 5 platform
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()
    exit()

login = 48438229  # Your account number
password = "Mq3p8!tJ"  # Your account password
server = "HFMarketsGlobal-Demo"  # Your broker's server

# Login to the trading account
if not mt5.login(login, password, server):
    print(f"Failed to connect to account {login}")
    mt5.shutdown()
    exit()

# Ensure AutoTrading is enabled
account_info = mt5.account_info()
if account_info is None:
    print("Failed to get account info, AutoTrading might be disabled")
    mt5.shutdown()
    exit()
elif not account_info.trade_allowed:
    print("AutoTrading is disabled, please enable it in the MetaTrader 5 platform")
    mt5.shutdown()
    exit()

# List of symbols
symbols = ["EURUSD", "GBPUSD", "USDJPY", "XAUUSD"]

# Set the specific dates and times to place the orders
scheduled_times = [
    datetime(2024, 6, 27, 12, 52, 0),  # First timer
    datetime(2024, 6, 27, 12, 55, 0),  # Second timer
    datetime(2024, 6, 27, 12, 59, 0)   # Third timer
]

# Define the lot size and pip parameters
lot = 1.0
pip_dict = {"EURUSD": 0.0001, "GBPUSD": 0.0001, "USDJPY": 0.01, "XAUUSD": 0.1}  # Different pip values for different symbols
stop_loss_pips = 10
trigger_pips = 20
trailing_pips = 15
stop_pips = 10

# Function to ensure the symbol is available
def ensure_symbol(symbol):
    symbol_info = mt5.symbol_info(symbol)
    if symbol_info is None:
        print(f"Symbol {symbol} not found, cannot call order_check()")
        return False
    elif not symbol_info.visible:
        print(f"Symbol {symbol} is not visible, trying to switch on")
        if not mt5.symbol_select(symbol, True):
            print(f"Symbol {symbol} is not available for trading")
            return False
    return True

# Function to place the orders
def place_orders(symbol):
    if not ensure_symbol(symbol):
        return

    pip = pip_dict[symbol]
    symbol_tick = mt5.symbol_info_tick(symbol)
    ask_price = symbol_tick.ask
    bid_price = symbol_tick.bid

    stop_loss_price_buy = ask_price - stop_loss_pips * pip
    stop_price_buy = ask_price + stop_pips * pip

    stop_loss_price_sell = bid_price + stop_loss_pips * pip
    stop_price_sell = bid_price - stop_pips * pip

    buy_stop_request = {
        "action": mt5.TRADE_ACTION_PENDING,
        "symbol": symbol,
        "volume": lot,
        "type": mt5.ORDER_TYPE_BUY_STOP,
        "price": stop_price_buy,
        "sl": stop_loss_price_buy,
        "deviation": 10,
        "magic": 234000,
        "comment": "python script buy stop",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_FOK,  # Using FOK filling mode
    }

    sell_stop_request = {
        "action": mt5.TRADE_ACTION_PENDING,
        "symbol": symbol,
        "volume": lot,
        "type": mt5.ORDER_TYPE_SELL_STOP,
        "price": stop_price_sell,
        "sl": stop_loss_price_sell,
        "deviation": 10,
        "magic": 234000,
        "comment": "python script sell stop",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_FOK,  # Using FOK filling mode
    }

    result_buy_stop = mt5.order_send(buy_stop_request)
    if result_buy_stop.retcode != mt5.TRADE_RETCODE_DONE:
        print(f"Buy stop order for {symbol} failed, retcode={result_buy_stop.retcode}")
        print(f"Error: {result_buy_stop}")
    else:
        print(f"Buy stop order for {symbol} placed successfully, ticket={result_buy_stop.order}")

    result_sell_stop = mt5.order_send(sell_stop_request)
    if result_sell_stop.retcode != mt5.TRADE_RETCODE_DONE:
        print(f"Sell stop order for {symbol} failed, retcode={result_sell_stop.retcode}")
        print(f"Error: {result_sell_stop}")
    else:
        print(f"Sell stop order for {symbol} placed successfully, ticket={result_sell_stop.order}")

# Function to close pending orders not executed within 3 minutes
def close_pending_orders(symbol, after_time):
    close_time = after_time + timedelta(minutes=50)
    while datetime.now() < close_time:
        time.sleep(1)

    orders = mt5.orders_get(symbol=symbol)
    if orders is None:
        print(f"No orders to close for {symbol}.")
        return

    for order in orders:
        if order.type in [mt5.ORDER_TYPE_BUY_STOP, mt5.ORDER_TYPE_SELL_STOP]:
            close_request = {
                "action": mt5.TRADE_ACTION_REMOVE,
                "order": order.ticket,
                "symbol": order.symbol,
                "volume": order.volume_current,
                "magic": order.magic,
                "comment": "python script cancel pending",
            }
            result = mt5.order_send(close_request)
            if result.retcode != mt5.TRADE_RETCODE_DONE:
                print(f"Failed to close order {order.ticket} for {symbol}, retcode={result.retcode}")
            else:
                print(f"Order {order.ticket} for {symbol} closed successfully.")

# Function to display the countdown timer
def display_countdown(target_time):
    while datetime.now() < target_time:
        now = datetime.now()
        remaining_time = target_time - now
        hours, remainder = divmod(remaining_time.seconds, 3600)
        minutes, seconds = divmod(remainder, 60)
        print(f"Time remaining: {remaining_time.days} days, {hours:02}:{minutes:02}:{seconds:02}", end="\r")
        time.sleep(1)

# Function to monitor and adjust trailing stops
def monitor_trailing_stops(symbol, target_time):
    pip = pip_dict[symbol]
    trigger_price_buy = None
    trigger_price_sell = None

    while datetime.now() < target_time + timedelta(minutes=30):
        positions = mt5.positions_get(symbol=symbol)
        if positions:
            for pos in positions:
                if pos.type == mt5.ORDER_TYPE_BUY:
                    if trigger_price_buy is None:
                        trigger_price_buy = pos.price_open + trigger_pips * pip
                    if mt5.symbol_info_tick(symbol).ask > trigger_price_buy:
                        new_sl = mt5.symbol_info_tick(symbol).ask - trailing_pips * pip
                        if new_sl > pos.sl:
                            request = {
                                "action": mt5.TRADE_ACTION_SLTP,
                                "position": pos.ticket,
                                "sl": new_sl,
                                "tp": pos.tp
                            }
                            result = mt5.order_send(request)
                            if result.retcode != mt5.TRADE_RETCODE_DONE:
                                print(f"Failed to modify SL for {symbol}, retcode={result.retcode}")
                            else:
                                print(f"Modified SL for {symbol} position {pos.ticket}")

                elif pos.type == mt5.ORDER_TYPE_SELL:
                    if trigger_price_sell is None:
                        trigger_price_sell = pos.price_open - trigger_pips * pip
                    if mt5.symbol_info_tick(symbol).bid < trigger_price_sell:
                        new_sl = mt5.symbol_info_tick(symbol).bid + trailing_pips * pip
                        if new_sl < pos.sl:
                            request = {
                                "action": mt5.TRADE_ACTION_SLTP,
                                "position": pos.ticket,
                                "sl": new_sl,
                                "tp": pos.tp
                            }
                            result = mt5.order_send(request)
                            if result.retcode != mt5.TRADE_RETCODE_DONE:
                                print(f"Failed to modify SL for {symbol}, retcode={result.retcode}")
                            else:
                                print(f"Modified SL for {symbol} position {pos.ticket}")

        time.sleep(1)

# Function to manage multiple timers for multiple symbols
def manage_timers(symbols, scheduled_times):
    for scheduled_time in scheduled_times:
        
        # Display the countdown timer
        display_countdown(scheduled_time)

        # Place the orders for each symbol
        for symbol in symbols:
            place_orders(symbol)

        # Close pending orders not taken after 3 minutes for each symbol
        for symbol in symbols:
            close_pending_orders(symbol, scheduled_time)

        # Monitor and adjust trailing stops for each symbol
        for symbol in symbols:
            monitor_trailing_stops(symbol, scheduled_time)

# Manage multiple timers for multiple symbols
manage_timers(symbols, scheduled_times)

# Shutdown the MetaTrader 5 platform
mt5.shutdown()


Buy stop order for EURUSD placed successfully, ticket=85678143
Sell stop order for EURUSD placed successfully, ticket=85678144
Buy stop order for GBPUSD placed successfully, ticket=85678145
Sell stop order for GBPUSD placed successfully, ticket=85678149
Buy stop order for USDJPY placed successfully, ticket=85678153
Sell stop order for USDJPY placed successfully, ticket=85678157
Buy stop order for XAUUSD placed successfully, ticket=85678163
Sell stop order for XAUUSD placed successfully, ticket=85678167
Order 85678144 for EURUSD closed successfully.
Order 85678149 for GBPUSD closed successfully.
Order 85678153 for USDJPY closed successfully.
Order 85678167 for XAUUSD closed successfully.
Buy stop order for EURUSD placed successfully, ticket=85719657
Sell stop order for EURUSD placed successfully, ticket=85719666
Buy stop order for GBPUSD placed successfully, ticket=85719672
Sell stop order for GBPUSD placed successfully, ticket=85719674
Buy stop order for USDJPY placed successfully, tic

True

In [ ]:
# Version 2 of the above, a possible fix of trailing stop and trade 2 n 3 not having countdown or executing

In [6]:
import MetaTrader5 as mt5
import time
from datetime import datetime, timedelta

# Initialize the MetaTrader 5 platform
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()
    exit()

login = 48438229  # Your account number
password = "Mq3p8!tJ"  # Your account password
server = "HFMarketsGlobal-Demo"  # Your broker's server

# Login to the trading account
if not mt5.login(login, password, server):
    print(f"Failed to connect to account {login}")
    mt5.shutdown()
    exit()

# Ensure AutoTrading is enabled
account_info = mt5.account_info()
if account_info is None:
    print("Failed to get account info, AutoTrading might be disabled")
    mt5.shutdown()
    exit()
elif not account_info.trade_allowed:
    print("AutoTrading is disabled, please enable it in the MetaTrader 5 platform")
    mt5.shutdown()
    exit()

# List of symbols
symbols = ["EURUSD", "GBPUSD", "AUDUSD", "GBPJPY", "USDJPY", "XAUUSD"]

# Set the specific dates and times to place the orders
scheduled_times = [
    datetime(2024, 6, 28, 16, 1, 0),  # First timer
    datetime(2024, 6, 28, 14, 28, 0),  # Second timer
    datetime(2024, 6, 28, 15, 29, 0)   # Third timer
]

# Define the lot size and pip parameters
lot = 1.0
pip_dict = {"EURUSD": 0.0001, "GBPUSD": 0.0001, "AUDUSD": 0.0001, "GBPJPY": 0.01, "USDJPY": 0.01, "XAUUSD": 0.1}  # Different pip values for different symbols
stop_loss_pips = 10
trigger_pips = 20
trailing_pips = 15
stop_pips = 10

# Function to ensure the symbol is available
def ensure_symbol(symbol):
    symbol_info = mt5.symbol_info(symbol)
    if symbol_info is None:
        print(f"Symbol {symbol} not found, cannot call order_check()")
        return False
    elif not symbol_info.visible:
        print(f"Symbol {symbol} is not visible, trying to switch on")
        if not mt5.symbol_select(symbol, True):
            print(f"Symbol {symbol} is not available for trading")
            return False
    return True

# Function to place the orders
def place_orders(symbol):
    if not ensure_symbol(symbol):
        return

    pip = pip_dict[symbol]
    symbol_tick = mt5.symbol_info_tick(symbol)
    ask_price = symbol_tick.ask
    bid_price = symbol_tick.bid

    stop_loss_price_buy = ask_price - stop_loss_pips * pip
    stop_price_buy = ask_price + stop_pips * pip

    stop_loss_price_sell = bid_price + stop_loss_pips * pip
    stop_price_sell = bid_price - stop_pips * pip

    buy_stop_request = {
        "action": mt5.TRADE_ACTION_PENDING,
        "symbol": symbol,
        "volume": lot,
        "type": mt5.ORDER_TYPE_BUY_STOP,
        "price": stop_price_buy,
        "sl": stop_loss_price_buy,
        "deviation": 10,
        "magic": 234000,
        "comment": "python script buy stop",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_FOK,  # Using FOK filling mode
    }

    sell_stop_request = {
        "action": mt5.TRADE_ACTION_PENDING,
        "symbol": symbol,
        "volume": lot,
        "type": mt5.ORDER_TYPE_SELL_STOP,
        "price": stop_price_sell,
        "sl": stop_loss_price_sell,
        "deviation": 10,
        "magic": 234000,
        "comment": "python script sell stop",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_FOK,  # Using FOK filling mode
    }

    result_buy_stop = mt5.order_send(buy_stop_request)
    if result_buy_stop.retcode != mt5.TRADE_RETCODE_DONE:
        print(f"Buy stop order for {symbol} failed, retcode={result_buy_stop.retcode}")
        print(f"Error: {result_buy_stop}")
    else:
        print(f"Buy stop order for {symbol} placed successfully, ticket={result_buy_stop.order}")

    result_sell_stop = mt5.order_send(sell_stop_request)
    if result_sell_stop.retcode != mt5.TRADE_RETCODE_DONE:
        print(f"Sell stop order for {symbol} failed, retcode={result_sell_stop.retcode}")
        print(f"Error: {result_sell_stop}")
    else:
        print(f"Sell stop order for {symbol} placed successfully, ticket={result_sell_stop.order}")

# Function to close pending orders not executed within 3 minutes
def close_pending_orders(symbol, after_time):
    close_time = after_time + timedelta(minutes=50)
    while datetime.now() < close_time:
        time.sleep(1)

    orders = mt5.orders_get(symbol=symbol)
    if orders is None:
        print(f"No orders to close for {symbol}.")
        return

    for order in orders:
        if order.type in [mt5.ORDER_TYPE_BUY_STOP, mt5.ORDER_TYPE_SELL_STOP]:
            close_request = {
                "action": mt5.TRADE_ACTION_REMOVE,
                "order": order.ticket,
                "symbol": order.symbol,
                "volume": order.volume_current,
                "magic": order.magic,
                "comment": "python script cancel pending",
            }
            result = mt5.order_send(close_request)
            if result.retcode != mt5.TRADE_RETCODE_DONE:
                print(f"Failed to close order {order.ticket} for {symbol}, retcode={result.retcode}")
            else:
                print(f"Order {order.ticket} for {symbol} closed successfully.")

# Function to display the countdown timer
def display_countdown(target_time):
    while datetime.now() < target_time:
        now = datetime.now()
        remaining_time = target_time - now
        hours, remainder = divmod(remaining_time.seconds, 3600)
        minutes, seconds = divmod(remainder, 60)
        print(f"Time remaining: {remaining_time.days} days, {hours:02}:{minutes:02}:{seconds:02}", end="\r")
        time.sleep(1)

# Function to monitor and adjust trailing stops
def monitor_trailing_stops(symbol, target_time):
    pip = pip_dict[symbol]
    trigger_price_buy = None
    trigger_price_sell = None

    while datetime.now() < target_time + timedelta(minutes=30):
        positions = mt5.positions_get(symbol=symbol)
        if positions:
            for pos in positions:
                if pos.type == mt5.ORDER_TYPE_BUY:
                    if trigger_price_buy is None:
                        trigger_price_buy = pos.price_open + trigger_pips * pip
                    if mt5.symbol_info_tick(symbol).ask > trigger_price_buy:
                        new_sl = mt5.symbol_info_tick(symbol).ask - trailing_pips * pip
                        if new_sl > pos.sl:
                            request = {
                                "action": mt5.TRADE_ACTION_SLTP,
                                "position": pos.ticket,
                                "sl": new_sl,
                                "tp": pos.tp
                            }
                            result = mt5.order_send(request)
                            if result.retcode != mt5.TRADE_RETCODE_DONE:
                                print(f"Failed to modify SL for {symbol}, retcode={result.retcode}")
                            else:
                                print(f"Modified SL for {symbol} position {pos.ticket}")

                elif pos.type == mt5.ORDER_TYPE_SELL:
                    if trigger_price_sell is None:
                        trigger_price_sell = pos.price_open - trigger_pips * pip
                    if mt5.symbol_info_tick(symbol).bid < trigger_price_sell:
                        new_sl = mt5.symbol_info_tick(symbol).bid + trailing_pips * pip
                        if new_sl < pos.sl:
                            request = {
                                "action": mt5.TRADE_ACTION_SLTP,
                                "position": pos.ticket,
                                "sl": new_sl,
                                "tp": pos.tp
                            }
                            result = mt5.order_send(request)
                            if result.retcode != mt5.TRADE_RETCODE_DONE:
                                print(f"Failed to modify SL for {symbol}, retcode={result.retcode}")
                            else:
                                print(f"Modified SL for {symbol} position {pos.ticket}")

        time.sleep(1)

# Function to manage multiple timers for multiple symbols
def manage_timers(symbols, scheduled_times):
    for scheduled_time in scheduled_times:
        
        # Display the countdown timer
        display_countdown(scheduled_time)

        # Place the orders for each symbol
        for symbol in symbols:
            place_orders(symbol)

        # Monitor and adjust trailing stops for each symbol
        for symbol in symbols:
            monitor_trailing_stops(symbol, scheduled_time)

        # Close pending orders not taken after 3 minutes for each symbol
        for symbol in symbols:
            close_pending_orders(symbol, scheduled_time)

# Manage multiple timers for multiple symbols
manage_timers(symbols, scheduled_times)

# Shutdown the MetaTrader 5 platform
mt5.shutdown()


Buy stop order for EURUSD placed successfully, ticket=86048337
Sell stop order for EURUSD placed successfully, ticket=86048338
Buy stop order for GBPUSD placed successfully, ticket=86048341
Sell stop order for GBPUSD placed successfully, ticket=86048342
Buy stop order for AUDUSD placed successfully, ticket=86048344
Sell stop order for AUDUSD placed successfully, ticket=86048345
Buy stop order for GBPJPY placed successfully, ticket=86048346
Sell stop order for GBPJPY placed successfully, ticket=86048347
Buy stop order for USDJPY placed successfully, ticket=86048348
Sell stop order for USDJPY placed successfully, ticket=86048350
Buy stop order for XAUUSD placed successfully, ticket=86048351
Sell stop order for XAUUSD placed successfully, ticket=86048352
Order 86048337 for EURUSD closed successfully.
Order 86048338 for EURUSD closed successfully.
Order 86048341 for GBPUSD closed successfully.
Order 86048342 for GBPUSD closed successfully.
Order 86048344 for AUDUSD closed successfully.
Ord

True

In [1]:
#Fix for above meta

In [7]:
import MetaTrader5 as mt5
import time
from datetime import datetime, timedelta

# Initialize the MetaTrader 5 platform
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()
    exit()

login = 48438229  # Your account number
password = "Mq3p8!tJ"  # Your account password
server = "HFMarketsGlobal-Demo"  # Your broker's server

# Login to the trading account
if not mt5.login(login, password, server):
    print(f"Failed to connect to account {login}")
    mt5.shutdown()
    exit()

# Ensure AutoTrading is enabled
account_info = mt5.account_info()
if account_info is None:
    print("Failed to get account info, AutoTrading might be disabled")
    mt5.shutdown()
    exit()
elif not account_info.trade_allowed:
    print("AutoTrading is disabled, please enable it in the MetaTrader 5 platform")
    mt5.shutdown()
    exit()

# List of symbols
symbols = ["EURUSD", "GBPUSD", "AUDUSD", "GBPJPY", "USDJPY", "XAUUSD"]

# Set the specific dates and times to place the orders
scheduled_times = [
    datetime(2024, 7, 1, 15, 52, 0),  # First timer
    datetime(2024, 7, 1, 15, 55, 0),  # Second timer
    datetime(2024, 7, 1, 15, 58, 0)   # Third timer
]

# Define the lot size and pip parameters
lot = 1.0
pip_dict = {"EURUSD": 0.0001, "GBPUSD": 0.0001, "AUDUSD": 0.0001, "GBPJPY": 0.01, "USDJPY": 0.01, "XAUUSD": 0.1}  # Different pip values for different symbols
stop_loss_pips = 10
trigger_pips = 20
trailing_pips = 15
stop_pips = 10

# Function to ensure the symbol is available
def ensure_symbol(symbol):
    symbol_info = mt5.symbol_info(symbol)
    if symbol_info is None:
        print(f"Symbol {symbol} not found, cannot call order_check()")
        return False
    elif not symbol_info.visible:
        print(f"Symbol {symbol} is not visible, trying to switch on")
        if not mt5.symbol_select(symbol, True):
            print(f"Symbol {symbol} is not available for trading")
            return False
    return True

# Function to place the orders
def place_orders(symbol):
    if not ensure_symbol(symbol):
        return

    pip = pip_dict[symbol]
    symbol_tick = mt5.symbol_info_tick(symbol)
    ask_price = symbol_tick.ask
    bid_price = symbol_tick.bid

    stop_loss_price_buy = ask_price - stop_loss_pips * pip
    stop_price_buy = ask_price + stop_pips * pip

    stop_loss_price_sell = bid_price + stop_loss_pips * pip
    stop_price_sell = bid_price - stop_pips * pip

    buy_stop_request = {
        "action": mt5.TRADE_ACTION_PENDING,
        "symbol": symbol,
        "volume": lot,
        "type": mt5.ORDER_TYPE_BUY_STOP,
        "price": stop_price_buy,
        "sl": stop_loss_price_buy,
        "deviation": 10,
        "magic": 234000,
        "comment": "python script buy stop",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_FOK,  # Using FOK filling mode
    }

    sell_stop_request = {
        "action": mt5.TRADE_ACTION_PENDING,
        "symbol": symbol,
        "volume": lot,
        "type": mt5.ORDER_TYPE_SELL_STOP,
        "price": stop_price_sell,
        "sl": stop_loss_price_sell,
        "deviation": 10,
        "magic": 234000,
        "comment": "python script sell stop",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_FOK,  # Using FOK filling mode
    }

    result_buy_stop = mt5.order_send(buy_stop_request)
    if result_buy_stop.retcode != mt5.TRADE_RETCODE_DONE:
        print(f"Buy stop order for {symbol} failed, retcode={result_buy_stop.retcode}")
        print(f"Error: {result_buy_stop}")
    else:
        print(f"Buy stop order for {symbol} placed successfully, ticket={result_buy_stop.order}")

    result_sell_stop = mt5.order_send(sell_stop_request)
    if result_sell_stop.retcode != mt5.TRADE_RETCODE_DONE:
        print(f"Sell stop order for {symbol} failed, retcode={result_sell_stop.retcode}")
        print(f"Error: {result_sell_stop}")
    else:
        print(f"Sell stop order for {symbol} placed successfully, ticket={result_sell_stop.order}")

# Function to close pending orders not executed within 3 minutes
def close_pending_orders(symbol, after_time):
    close_time = after_time + timedelta(minutes=50)
    while datetime.now() < close_time:
        time.sleep(1)

    orders = mt5.orders_get(symbol=symbol)
    if orders is None:
        print(f"No orders to close for {symbol}.")
        return

    for order in orders:
        if order.type in [mt5.ORDER_TYPE_BUY_STOP, mt5.ORDER_TYPE_SELL_STOP]:
            close_request = {
                "action": mt5.TRADE_ACTION_REMOVE,
                "order": order.ticket,
                "symbol": order.symbol,
                "volume": order.volume_current,
                "magic": order.magic,
                "comment": "python script cancel pending",
            }
            result = mt5.order_send(close_request)
            if result.retcode != mt5.TRADE_RETCODE_DONE:
                print(f"Failed to close order {order.ticket} for {symbol}, retcode={result.retcode}")
            else:
                print(f"Order {order.ticket} for {symbol} closed successfully.")

# Function to display the countdown timer
def display_countdown(target_time):
    while datetime.now() < target_time:
        now = datetime.now()
        remaining_time = target_time - now
        hours, remainder = divmod(remaining_time.seconds, 3600)
        minutes, seconds = divmod(remainder, 60)
        print(f"Time remaining: {remaining_time.days} days, {hours:02}:{minutes:02}:{seconds:02}", end="\r")
        time.sleep(1)

# ... (rest of the code remains the same)

# Function to monitor and adjust trailing stops
def monitor_trailing_stops(symbol, target_time):
    pip = pip_dict[symbol]
    trigger_price_buy = None
    trigger_price_sell = None
    position = mt5.positions_get(symbol=symbol)
    
    if position:
        pos = position[0]
        if pos.type == mt5.ORDER_TYPE_BUY:
            if trigger_price_buy is None:
                trigger_price_buy = pos.price_open + trigger_pips * pip
            if mt5.symbol_info_tick(symbol).ask > trigger_price_buy:
                new_sl = mt5.symbol_info_tick(symbol).ask - trailing_pips * pip
                if new_sl > pos.sl:
                    request = {
                        "action": mt5.TRADE_ACTION_SLTP,
                        "position": pos.ticket,
                        "sl": new_sl,
                        "tp": pos.tp
                    }
                    result = mt5.order_send(request)
                    if result.retcode != mt5.TRADE_RETCODE_DONE:
                        print(f"Failed to modify SL for {symbol}, retcode={result.retcode}")
                    else:
                        print(f"Modified SL for {symbol} position {pos.ticket}")
        elif pos.type == mt5.ORDER_TYPE_SELL:
            if trigger_price_sell is None:
                trigger_price_sell = pos.price_open - trigger_pips * pip
            if mt5.symbol_info_tick(symbol).bid < trigger_price_sell:
                new_sl = mt5.symbol_info_tick(symbol).bid + trailing_pips * pip
                if new_sl < pos.sl:
                    request = {
                        "action": mt5.TRADE_ACTION_SLTP,
                        "position": pos.ticket,
                        "sl": new_sl,
                        "tp": pos.tp
                    }
                    result = mt5.order_send(request)
                    if result.retcode != mt5.TRADE_RETCODE_DONE:
                        print(f"Failed to modify SL for {symbol}, retcode={result.retcode}")
                    else:
                        print(f"Modified SL for {symbol} position {pos.ticket}")
    else:
        print(f"No position found for {symbol}")

# ... (rest of the code remains the same)

# Function to manage multiple timers for multiple symbols
def manage_timers(symbols, scheduled_times):
    for scheduled_time in scheduled_times:
        # Display the countdown timer
        print(f"Countdown to {scheduled_time}")
        display_countdown(scheduled_time)

        # Place the orders for each symbol
        for symbol in symbols:
            place_orders(symbol)

        # Monitor and adjust trailing stops for each symbol
        for symbol in symbols:
            monitor_trailing_stops(symbol, scheduled_time)

        # Close pending orders not taken after 3 minutes for each symbol
        for symbol in symbols:
            close_pending_orders(symbol, scheduled_time)
            
# Manage multiple timers for multiple symbols
manage_timers(symbols, scheduled_times)

# Shutdown the MetaTrader 5 platform
mt5.shutdown()


Countdown to 2024-07-01 15:52:00
Buy stop order for EURUSD failed, retcode=10031
Error: OrderSendResult(retcode=10031, deal=0, order=0, volume=0.0, price=0.0, bid=0.0, ask=0.0, comment='Request rejected due to absence of network connection', request_id=0, retcode_external=0, request=TradeRequest(action=5, magic=234000, order=0, symbol='EURUSD', volume=1.0, price=1.0745999999999998, stoplimit=0.0, sl=1.0726, tp=0.0, deviation=10, type=4, type_filling=0, type_time=0, expiration=0, comment='python script buy stop', position=0, position_by=0))
Sell stop order for EURUSD failed, retcode=10031
Error: OrderSendResult(retcode=10031, deal=0, order=0, volume=0.0, price=0.0, bid=0.0, ask=0.0, comment='Request rejected due to absence of network connection', request_id=0, retcode_external=0, request=TradeRequest(action=5, magic=234000, order=0, symbol='EURUSD', volume=1.0, price=1.0724300000000002, stoplimit=0.0, sl=1.07443, tp=0.0, deviation=10, type=5, type_filling=0, type_time=0, expiration=0, c

Sell stop order for USDJPY placed successfully, ticket=86338654
Buy stop order for XAUUSD placed successfully, ticket=86338655
Sell stop order for XAUUSD placed successfully, ticket=86338657
No position found for GBPUSD
No position found for AUDUSD
No position found for USDJPY
Modified SL for XAUUSD position 86236723
Order 86338645 for EURUSD closed successfully.
Order 86338646 for EURUSD closed successfully.
Order 86338647 for GBPUSD closed successfully.
Order 86338648 for GBPUSD closed successfully.
Order 86338649 for AUDUSD closed successfully.
Order 86338650 for AUDUSD closed successfully.
Order 86338651 for GBPJPY closed successfully.
Order 86338652 for GBPJPY closed successfully.
Order 86338653 for USDJPY closed successfully.
Order 86338654 for USDJPY closed successfully.
Order 86338657 for XAUUSD closed successfully.


True

In [3]:
#gpt fix

In [4]:
# Seems to work....Trailing stop still having SL of 20 pips tho instead of 10 pips then Trailing stop is like 40 pips
# Cuz gold was on 1,200 but ended up on 791
# Trades taken at 3 different times i guess, deletion too working i guess

#So after testing again, Trailing stop is the only issue, timer or 3 and all is working and it seems the trailingstop thing is affecting the SL too


In [3]:
import MetaTrader5 as mt5
import time
from datetime import datetime, timedelta

# Initialize the MetaTrader 5 platform
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()
    exit()

login = 48438229  # Your account number
password = "Mq3p8!tJ"  # Your account password
server = "HFMarketsGlobal-Demo"  # Your broker's server

# Login to the trading account
if not mt5.login(login, password, server):
    print(f"Failed to connect to account {login}")
    mt5.shutdown()
    exit()

# Ensure AutoTrading is enabled
account_info = mt5.account_info()
if account_info is None:
    print("Failed to get account info, AutoTrading might be disabled")
    mt5.shutdown()
    exit()
elif not account_info.trade_allowed:
    print("AutoTrading is disabled, please enable it in the MetaTrader 5 platform")
    mt5.shutdown()
    exit()

# List of symbols
symbols = ["EURUSD", "GBPUSD", "AUDUSD", "GBPJPY", "USDJPY", "XAUUSD"]

# Set the specific dates and times to place the orders
scheduled_times = [
    datetime(2024, 7, 3, 13, 28, 0),  # First timer
    datetime(2024, 7, 3, 14, 58, 0),  # Second timer
    datetime(2024, 7, 3, 18, 58, 0)   # Third timer
]

# Define the lot size and pip parameters
lot = 1.0
pip_dict = {"EURUSD": 0.0001, "GBPUSD": 0.0001, "AUDUSD": 0.0001, "GBPJPY": 0.01, "USDJPY": 0.01, "XAUUSD": 0.1}  # Different pip values for different symbols
stop_loss_pips = 10
trigger_pips = 10
trailing_pips = 15
stop_pips = 10

# Function to ensure the symbol is available
def ensure_symbol(symbol):
    symbol_info = mt5.symbol_info(symbol)
    if symbol_info is None:
        print(f"Symbol {symbol} not found, cannot call order_check()")
        return False
    elif not symbol_info.visible:
        print(f"Symbol {symbol} is not visible, trying to switch on")
        if not mt5.symbol_select(symbol, True):
            print(f"Symbol {symbol} is not available for trading")
            return False
    return True

# Function to place the orders
def place_orders(symbol):
    if not ensure_symbol(symbol):
        return

    pip = pip_dict[symbol]
    symbol_tick = mt5.symbol_info_tick(symbol)
    ask_price = symbol_tick.ask
    bid_price = symbol_tick.bid

    stop_loss_price_buy = ask_price - stop_loss_pips * pip
    stop_price_buy = ask_price + stop_pips * pip

    stop_loss_price_sell = bid_price + stop_loss_pips * pip
    stop_price_sell = bid_price - stop_pips * pip

    buy_stop_request = {
        "action": mt5.TRADE_ACTION_PENDING,
        "symbol": symbol,
        "volume": lot,
        "type": mt5.ORDER_TYPE_BUY_STOP,
        "price": stop_price_buy,
        "sl": stop_loss_price_buy,
        "deviation": 10,
        "magic": 234000,
        "comment": "python script buy stop",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_FOK,  # Using FOK filling mode
    }

    sell_stop_request = {
        "action": mt5.TRADE_ACTION_PENDING,
        "symbol": symbol,
        "volume": lot,
        "type": mt5.ORDER_TYPE_SELL_STOP,
        "price": stop_price_sell,
        "sl": stop_loss_price_sell,
        "deviation": 10,
        "magic": 234000,
        "comment": "python script sell stop",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_FOK,  # Using FOK filling mode
    }

    result_buy_stop = mt5.order_send(buy_stop_request)
    if result_buy_stop.retcode != mt5.TRADE_RETCODE_DONE:
        print(f"Buy stop order for {symbol} failed, retcode={result_buy_stop.retcode}")
        print(f"Error: {result_buy_stop}")
    else:
        print(f"Buy stop order for {symbol} placed successfully, ticket={result_buy_stop.order}")

    result_sell_stop = mt5.order_send(sell_stop_request)
    if result_sell_stop.retcode != mt5.TRADE_RETCODE_DONE:
        print(f"Sell stop order for {symbol} failed, retcode={result_sell_stop.retcode}")
        print(f"Error: {result_sell_stop}")
    else:
        print(f"Sell stop order for {symbol} placed successfully, ticket={result_sell_stop.order}")

# Function to close pending orders not executed within 3 minutes
def close_pending_orders(symbol, after_time):
    close_time = after_time + timedelta(minutes=5)
    while datetime.now() < close_time:
        time.sleep(1)

    orders = mt5.orders_get(symbol=symbol)
    if orders is None:
        print(f"No orders to close for {symbol}.")
        return

    for order in orders:
        if order.type in [mt5.ORDER_TYPE_BUY_STOP, mt5.ORDER_TYPE_SELL_STOP]:
            close_request = {
                "action": mt5.TRADE_ACTION_REMOVE,
                "order": order.ticket,
                "symbol": order.symbol,
                "volume": order.volume_current,
                "magic": order.magic,
                "comment": "python script cancel pending",
            }
            result = mt5.order_send(close_request)
            if result.retcode != mt5.TRADE_RETCODE_DONE:
                print(f"Failed to close order {order.ticket} for {symbol}, retcode={result.retcode}")
            else:
                print(f"Order {order.ticket} for {symbol} closed successfully.")

# Function to display the countdown timer
def display_countdown(target_time):
    while datetime.now() < target_time:
        now = datetime.now()
        remaining_time = target_time - now
        hours, remainder = divmod(remaining_time.seconds, 3600)
        minutes, seconds = divmod(remainder, 60)
        print(f"Time remaining: {remaining_time.days} days, {hours:02}:{minutes:02}:{seconds:02}", end="\r")
        time.sleep(1)

        
# Function to monitor and adjust trailing stops
def monitor_trailing_stops(symbol, target_time):
    pip = pip_dict[symbol]
    while datetime.now() < target_time + timedelta(minutes=30):
        positions = mt5.positions_get(symbol=symbol)
        if positions:
            for pos in positions:
                if pos.type == mt5.ORDER_TYPE_BUY:
                    current_price = mt5.symbol_info_tick(symbol).ask
                    if current_price > pos.price_open + trigger_pips * pip:
                        new_sl = current_price - trailing_pips * pip
                        if new_sl > pos.sl:
                            request = {
                                "action": mt5.TRADE_ACTION_SLTP,
                                "position": pos.ticket,
                                "sl": new_sl,
                                "tp": pos.tp
                            }
                            result = mt5.order_send(request)
                            if result.retcode != mt5.TRADE_RETCODE_DONE:
                                print(f"Failed to modify SL for {symbol}, retcode={result.retcode}")
                            else:
                                print(f"Modified SL for {symbol} position {pos.ticket}")
                elif pos.type == mt5.ORDER_TYPE_SELL:
                    current_price = mt5.symbol_info_tick(symbol).bid
                    if current_price < pos.price_open - trigger_pips * pip:
                        new_sl = current_price + trailing_pips * pip
                        if new_sl < pos.sl:
                            request = {
                                "action": mt5.TRADE_ACTION_SLTP,
                                "position": pos.ticket,
                                "sl": new_sl,
                                "tp": pos.tp
                            }
                            result = mt5.order_send(request)
                            if result.retcode != mt5.TRADE_RETCODE_DONE:
                                print(f"Failed to modify SL for {symbol}, retcode={result.retcode}")
                            else:
                                print(f"Modified SL for {symbol} position {pos.ticket}")
        time.sleep(1)


# Function to manage multiple timers for multiple symbols
def manage_timers(symbols, scheduled_times):
    for scheduled_time in scheduled_times:
        # Display the countdown timer
        print(f"Countdown to {scheduled_time}")
        display_countdown(scheduled_time)

        # Place the orders for each symbol
        for symbol in symbols:
            place_orders(symbol)

        # Monitor and adjust trailing stops for each symbol
        for symbol in symbols:
            monitor_trailing_stops(symbol, scheduled_time)

        # Close pending orders not taken after 3 minutes for each symbol
        for symbol in symbols:
            close_pending_orders(symbol, scheduled_time)
            
# Manage multiple timers for multiple symbols
manage_timers(symbols, scheduled_times)

# Shutdown the MetaTrader 5 platform
mt5.shutdown()


Countdown to 2024-07-03 13:28:00
Buy stop order for EURUSD placed successfully, ticket=86800302
Sell stop order for EURUSD placed successfully, ticket=86800304
Buy stop order for GBPUSD placed successfully, ticket=86800307
Sell stop order for GBPUSD placed successfully, ticket=86800309
Buy stop order for AUDUSD placed successfully, ticket=86800313
Sell stop order for AUDUSD placed successfully, ticket=86800315
Buy stop order for GBPJPY placed successfully, ticket=86800318
Sell stop order for GBPJPY placed successfully, ticket=86800319
Buy stop order for USDJPY placed successfully, ticket=86800321
Sell stop order for USDJPY placed successfully, ticket=86800322
Buy stop order for XAUUSD placed successfully, ticket=86800323
Sell stop order for XAUUSD placed successfully, ticket=86800324
Order 86800304 for EURUSD closed successfully.
Order 86800309 for GBPUSD closed successfully.
Order 86800315 for AUDUSD closed successfully.
Order 86800319 for GBPJPY closed successfully.
Order 86800321 fo

True

In [ ]:
# Possible trailingstop fix for the direct above

In [3]:
import MetaTrader5 as mt5
import time
from datetime import datetime, timedelta

# Initialize the MetaTrader 5 platform
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()
    exit()

login = 48438229  # Your account number
password = "Mq3p8!tJ"  # Your account password
server = "HFMarketsGlobal-Demo"  # Your broker's server

# Login to the trading account
if not mt5.login(login, password, server):
    print(f"Failed to connect to account {login}")
    mt5.shutdown()
    exit()

# Ensure AutoTrading is enabled
account_info = mt5.account_info()
if account_info is None:
    print("Failed to get account info, AutoTrading might be disabled")
    mt5.shutdown()
    exit()
elif not account_info.trade_allowed:
    print("AutoTrading is disabled, please enable it in the MetaTrader 5 platform")
    mt5.shutdown()
    exit()

# List of symbols
symbols = ["EURUSD", "GBPUSD", "AUDUSD", "GBPJPY", "USDJPY", "XAUUSD"]

# Set the specific dates and times to place the orders
scheduled_times = [
    datetime(2024, 7, 5, 13, 28, 0),  # First timer
    #datetime(2024, 7, 3, 14, 58, 0),  # Second timer
    #datetime(2024, 7, 3, 18, 58, 0)   # Third timer
]

# Define the lot size and pip parameters
lot = 1.0
pip_dict = {"EURUSD": 0.0001, "GBPUSD": 0.0001, "AUDUSD": 0.0001, "GBPJPY": 0.01, "USDJPY": 0.01, "XAUUSD": 0.1}
trailing_pip = 15
trail_amount_update = 3
default_sl_pip = 10
bs_stop = 15
stop_loss = 10

# Function to ensure the symbol is available
def ensure_symbol(symbol):
    symbol_info = mt5.symbol_info(symbol)
    if symbol_info is None:
        print(f"Symbol {symbol} not found, cannot call order_check()")
        return False
    elif not symbol_info.visible:
        print(f"Symbol {symbol} is not visible, trying to switch on")
        if not mt5.symbol_select(symbol, True):
            print(f"Symbol {symbol} is not available for trading")
            return False
    return True

# Function to place the orders
def place_orders(symbol):
    if not ensure_symbol(symbol):
        return

    pip = pip_dict[symbol]
    symbol_tick = mt5.symbol_info_tick(symbol)
    ask_price = symbol_tick.ask
    bid_price = symbol_tick.bid

    stop_loss_price_buy = ask_price - stop_loss * pip
    stop_price_buy = ask_price + bs_stop * pip

    stop_loss_price_sell = bid_price + stop_loss * pip
    stop_price_sell = bid_price - bs_stop * pip

    buy_stop_request = {
        "action": mt5.TRADE_ACTION_PENDING,
        "symbol": symbol,
        "volume": lot,
        "type": mt5.ORDER_TYPE_BUY_STOP,
        "price": stop_price_buy,
        "sl": stop_loss_price_buy,
        "deviation": 10,
        "magic": 234000,
        "comment": "python script buy stop",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_FOK,
    }

    sell_stop_request = {
        "action": mt5.TRADE_ACTION_PENDING,
        "symbol": symbol,
        "volume": lot,
        "type": mt5.ORDER_TYPE_SELL_STOP,
        "price": stop_price_sell,
        "sl": stop_loss_price_sell,
        "deviation": 10,
        "magic": 234000,
        "comment": "python script sell stop",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_FOK,
    }

    result_buy_stop = mt5.order_send(buy_stop_request)
    if result_buy_stop.retcode != mt5.TRADE_RETCODE_DONE:
        print(f"Buy stop order for {symbol} failed, retcode={result_buy_stop.retcode}")
        print(f"Error: {result_buy_stop}")
    else:
        print(f"Buy stop order for {symbol} placed successfully, ticket={result_buy_stop.order}")

    result_sell_stop = mt5.order_send(sell_stop_request)
    if result_sell_stop.retcode != mt5.TRADE_RETCODE_DONE:
        print(f"Sell stop order for {symbol} failed, retcode={result_sell_stop.retcode}")
        print(f"Error: {result_sell_stop}")
    else:
        print(f"Sell stop order for {symbol} placed successfully, ticket={result_sell_stop.order}")

# Function to close pending orders not executed within 50 minutes
def close_pending_orders(symbol, after_time):
    close_time = after_time + timedelta(minutes=50)
    while datetime.now() < close_time:
        time.sleep(1)

    orders = mt5.orders_get(symbol=symbol)
    if orders is None:
        print(f"No orders to close for {symbol}.")
        return

    for order in orders:
        if order.type in [mt5.ORDER_TYPE_BUY_STOP, mt5.ORDER_TYPE_SELL_STOP]:
            close_request = {
                "action": mt5.TRADE_ACTION_REMOVE,
                "order": order.ticket,
                "symbol": order.symbol,
                "volume": order.volume_current,
                "magic": order.magic,
                "comment": "python script cancel pending",
            }
            result = mt5.order_send(close_request)
            if result.retcode != mt5.TRADE_RETCODE_DONE:
                print(f"Failed to close order {order.ticket} for {symbol}, retcode={result.retcode}")
            else:
                print(f"Order {order.ticket} for {symbol} closed successfully.")

# Function to display the countdown timer
def display_countdown(target_time):
    while datetime.now() < target_time:
        now = datetime.now()
        remaining_time = target_time - now
        hours, remainder = divmod(remaining_time.seconds, 3600)
        minutes, seconds = divmod(remainder, 60)
        print(f"Time remaining: {remaining_time.days} days, {hours:02}:{minutes:02}:{seconds:02}", end="\r")
        time.sleep(1)


def trail_sl(symbol):
    pip = pip_dict[symbol]
    MAX_DIST_SL = trailing_pip * pip
    TRAIL_AMOUNT = trail_amount_update * pip
    DEFAULT_SL = default_sl_pip * pip

    positions = mt5.positions_get(symbol=symbol)
    if not positions:
        print(f"No open positions for {symbol}")
        return

    for position in positions:
        order_type = position.type
        price_current = mt5.symbol_info_tick(symbol).ask if order_type == mt5.ORDER_TYPE_BUY else mt5.symbol_info_tick(symbol).bid
        price_open = position.price_open
        sl = position.sl

        dist_from_sl = abs(price_current - sl)

        if dist_from_sl > MAX_DIST_SL:
            if sl != 0.0:
                new_sl = sl + TRAIL_AMOUNT if order_type == mt5.ORDER_TYPE_BUY else sl - TRAIL_AMOUNT
            else:
                new_sl = price_open - DEFAULT_SL if order_type == mt5.ORDER_TYPE_BUY else price_open + DEFAULT_SL

            request = {
                'action': mt5.TRADE_ACTION_SLTP,
                'position': position.ticket,
                'sl': new_sl,
            }

            result = mt5.order_send(request)
            if result.retcode != mt5.TRADE_RETCODE_DONE:
                print(f"Failed to modify SL for {symbol} position {position.ticket}, retcode={result.retcode}")
                print(f"Error: {result}")
            else:
                print(f"Modified SL for {symbol} position {position.ticket}, new SL={new_sl}")

# Function to monitor and adjust trailing stops
def monitor_trailing_stops(symbol, target_time):
    end_time = target_time + timedelta(minutes=30)
    while datetime.now() < end_time:
        print(f"Checking trailing stops for {symbol}")
        trail_sl(symbol)
        time.sleep(1)

# Function to manage multiple timers for multiple symbols
def manage_timers(symbols, scheduled_times):
    for scheduled_time in scheduled_times:
        print(f"Countdown to {scheduled_time}")
        display_countdown(scheduled_time)

        for symbol in symbols:
            place_orders(symbol)

        for symbol in symbols:
            monitor_trailing_stops(symbol, scheduled_time)

        for symbol in symbols:
            close_pending_orders(symbol, scheduled_time)

# Manage multiple timers for multiple symbols
manage_timers(symbols, scheduled_times)

# Shutdown the MetaTrader 5 platform
mt5.shutdown()


Countdown to 2024-07-05 13:28:00
Buy stop order for EURUSD placed successfully, ticket=87257448
Sell stop order for EURUSD placed successfully, ticket=87257451
Buy stop order for GBPUSD placed successfully, ticket=87257456
Sell stop order for GBPUSD placed successfully, ticket=87257457
Buy stop order for AUDUSD placed successfully, ticket=87257464
Sell stop order for AUDUSD placed successfully, ticket=87257469
Buy stop order for GBPJPY placed successfully, ticket=87257471
Sell stop order for GBPJPY placed successfully, ticket=87257475
Buy stop order for USDJPY placed successfully, ticket=87257479
Sell stop order for USDJPY placed successfully, ticket=87257483
Buy stop order for XAUUSD placed successfully, ticket=87257487
Sell stop order for XAUUSD placed successfully, ticket=87257493
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trai

Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
Modified SL for EURUSD position 87257451, new SL=1.08307
Checking trailing stops for EURUSD
Modified SL for EURUSD position 87257451, new SL=1.08277
Checking trailing stops for EURUSD
Failed to modify SL for EURUSD position 87257451, retcode=10016
Error: OrderSendResult(retcode=10016, deal=0, order=0, volume=0.0, price=0.0, bid=0.0, ask=0.0, comment='Invalid stops', request_id=1595914030, retcode_external=0, request=TradeRequest(action=6, magic=0, order=0, symbol='', volume=0.0, price=0.0, stoplimit=0.0, sl=1.08247, tp=0.0, deviation=0, type=0, type_filling=0, type_time=0, e

Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No op

Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No op

Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No op

...

Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No op

Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No op

Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No op

Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No open positions for EURUSD
Checking trailing stops for EURUSD
No op

True

import MetaTrader5 as mt5

Initialize the MT5 API
mt5.initialize()

Set the symbol and lot size
symbol = "EURUSD"
lot_size = 0.1

Open a buy trade
result = mt5.order_send(symbol, mt5.OP_BUY, lot_size, mt5.ask, 3, mt5.ask-20*mt5.point, mt5.ask+20*mt5.point, "MyComment", 0, 0, mt5.GREEN)

Get the ticket number of the trade
ticket = result.order

Set a trailing stop of 20 points
mt5.trailing_stop_set(ticket, 20)

Shutdown the MT5 API
mt5.shutdown()

In [ ]:
# Function to update trailing stop if needed
def update_trailing_stop(order, trailing_stop_pips, pip_size):
    # Convert trailing_stop_pips into pips
    trailing_stop_pips = trailing_stop_pips * pip_size99
    # Determine if Red or Green
    # A Green Position will have a take_profit > stop_loss
    if order[12] > order[11]:
        # If Green, new_stop_loss = current_price - trailing_stop_pips
        new_stop_loss = order[13] - trailing_stop_pips
        # Test to see if new_stop_loss > current_stop_loss
        if new_stop_loss > order[11]:
            print("Update Stop Loss")
            # Create updated values for order
            order_number = order[0]
            symbol = order[16]
            # New take_profit will be the difference between new_stop_loss and old_stop_loss added to take profit
            new_take_profit = order[12] + new_stop_loss - order[11]
            print(new_take_profit)
            # Send order to modify_position
            mt5_interface.modify_position(order_number=order_number, symbol=symbol, new_stop_loss=new_stop_loss,
                                          new_take_profit=new_take_profit)
    elif order[12] < order[11]:
        # If Red, new_stop_loss = current_price + trailing_stop_pips
        new_stop_loss = order[13] + trailing_stop_pips
        # Test to see if new_stop_loss < current_stop_loss
        if new_stop_loss < order[11]:
            print("Update Stop Loss")
            # Create updated values for order
            order_number = order[0]
            symbol = order[16]
            # New take_profit will be the difference between new_stop_loss and old_stop_loss subtracted from old take_profit
            new_take_profit = order[12] - new_stop_loss + order[11]
            print(new_take_profit)
            # Send order to modify_position
            mt5_interface.modify_position(order_number=order_number, symbol=symbol, new_stop_loss=new_stop_loss,
                                          new_take_profit=new_take_profit)

In [ ]:
"""
Trailing Stoploss
Author: TraderPy

Youtube: https://www.youtube.com/channel/UC9xYCyyR_G3LIuJ_LlTiEVQ/featured

Website: https://traderpy.com/

Disclaimer
Trading the financial markets imposes a risk of financial loss. TraderPy is not responsible for any financial losses
that viewers suffer. Content is educational only and does not serve as financial advice. Information or material is
provided ‘as is’ without any warranty.

Past trading results do not indicate future performance. Strategies that worked in the past may not reflect the same
results in the future.
"""

# import libraries
import MetaTrader5 as mt5
import time
import sys

# connect Python to MetaTrader5
mt5.initialize()

# CONFIGS
TICKET = 165856631
MAX_DIST_SL = 0.0006  # Max distance between current price and SL, otherwise SL will update
TRAIL_AMOUNT = 0.0003  # Amount by how much SL updates
DEFAULT_SL = 0.0003  # If position has no SL, set a default SL


# function to trail SL
def trail_sl():
    # get position based on ticket_id
    position = mt5.positions_get(ticket=TICKET)

    # check if position exists
    if position:
        position = position[0]
    else:
        print('Position does not exist')
        sys.exit()

    # get position data
    order_type = position.type
    price_current = position.price_current
    price_open = position.price_open
    sl = position.sl

    dist_from_sl = abs(round(price_current - sl, 6))

    if dist_from_sl > MAX_DIST_SL:
        # calculating new sl
        if sl != 0.0:
            if order_type == 0:  # 0 stands for BUY
                new_sl = sl + TRAIL_AMOUNT

            elif order_type == 1:  # 1 stands for SELL
                new_sl = sl - TRAIL_AMOUNT

        else:
            # setting default SL if the is no SL on the symbol
            new_sl = price_open - DEFAULT_SL if order_type == 0 else price_open + DEFAULT_SL

        request = {
            'action': mt5.TRADE_ACTION_SLTP,
            'position': position.ticket,
            'sl': new_sl,
        }

        result = mt5.order_send(request)
        print(result)
        return result


if __name__ == '__main__':
    print('Starting Trailing Stoploss..')
    print(f'Position: {str(TICKET)}')

    # strategy loop
    while True:
        result = trail_sl()
        # wait 1 second
        time.sleep(1)





In [ ]:
#trailing stop from gpt on another chat n topic

In [ ]:
import MetaTrader5 as mt5

# Connect to the MetaTrader 5 terminal
mt5.initialize()

# Define the symbol and lot size
symbol = "EURUSD"
lot_size = 0.1

# Define the entry points and stop loss levels
buy_stop = mt5.symbol_info_tick(symbol).ask + 20 * mt5.symbol_info(symbol).point
sell_stop = mt5.symbol_info_tick(symbol).bid - 20 * mt5.symbol_info(symbol).point
stop_loss = 30 * mt5.symbol_info(symbol).point
take_profit = 50 * mt5.symbol_info(symbol).point

# Define the trailing stop parameters
trailing_stop = 10 * mt5.symbol_info(symbol).point
trailing_step = 3 * mt5.symbol_info(symbol).point

# Place buy stop order
buy_order = {
    "action": mt5.TRADE_ACTION_PENDING,
    "symbol": symbol,
    "volume": lot_size,
    "type": mt5.ORDER_TYPE_BUY_STOP,
    "price": buy_stop,
    "sl": buy_stop - stop_loss,
    "tp": buy_stop + take_profit,
    "deviation": 10,
    "magic": 234000,
    "comment": "Buy Stop Order",
    "type_time": mt5.ORDER_TIME_GTC,
    "type_filling": mt5.ORDER_FILLING_RETURN,
}

# Place sell stop order
sell_order = {
    "action": mt5.TRADE_ACTION_PENDING,
    "symbol": symbol,
    "volume": lot_size,
    "type": mt5.ORDER_TYPE_SELL_STOP,
    "price": sell_stop,
    "sl": sell_stop + stop_loss,
    "tp": sell_stop - take_profit,
    "deviation": 10,
    "magic": 234000,
    "comment": "Sell Stop Order",
    "type_time": mt5.ORDER_TIME_GTC,
    "type_filling": mt5.ORDER_FILLING_RETURN,
}

# Send the orders
buy_result = mt5.order_send(buy_order)
sell_result = mt5.order_send(sell_order)

# Check if orders were placed successfully
if buy_result.retcode != mt5.TRADE_RETCODE_DONE:
    print(f"Failed to place buy stop order: {buy_result.comment}")
if sell_result.retcode != mt5.TRADE_RETCODE_DONE:
    print(f"Failed to place sell stop order: {sell_result.comment}")

# Function to update trailing stop
def update_trailing_stop(order_ticket, trailing_stop, trailing_step):
    order = mt5.order_get(ticket=order_ticket)
    if order:
        current_price = mt5.symbol_info_tick(symbol).bid if order.type == mt5.ORDER_TYPE_SELL else mt5.symbol_info_tick(symbol).ask
        if order.type == mt5.ORDER_TYPE_SELL:
            new_sl = current_price + trailing_stop
            if new_sl < order.sl:
                request = {
                    "action": mt5.TRADE_ACTION_SLTP,
                    "symbol": symbol,
                    "sl": new_sl,
                    "ticket": order.ticket,
                }
                mt5.order_send(request)
        elif order.type == mt5.ORDER_TYPE_BUY:
            new_sl = current_price - trailing_stop
            if new_sl > order.sl:
                request = {
                    "action": mt5.TRADE_ACTION_SLTP,
                    "symbol": symbol,
                    "sl": new_sl,
                    "ticket": order.ticket,
                }
                mt5.order_send(request)

# Monitor and update trailing stops
while True:
    open_orders = mt5.orders_get(symbol=symbol)
    for order in open_orders:
        update_trailing_stop(order.ticket, trailing_stop, trailing_step)


In [1]:
# dynamic inputs parameters

In [ ]:
import MetaTrader5 as mt5
import time
from datetime import datetime, timedelta
import argparse

# Function to parse command-line arguments
def parse_arguments():
    parser = argparse.ArgumentParser(description='Forex trading bot with dynamic input parameters.')
    parser.add_argument('--symbols', type=str, nargs='+', required=True, help='List of symbols to trade')
    parser.add_argument('--times', type=str, nargs='+', required=True, help='List of scheduled times (YYYY-MM-DD HH:MM:SS)')
    parser.add_argument('--lot', type=float, required=True, help='Lot size')
    parser.add_argument('--stop_loss_pips', type=int, required=True, help='Stop loss in pips')
    parser.add_argument('--trigger_pips', type=int, required=True, help='Trigger pips for trailing stop')
    parser.add_argument('--trailing_pips', type=int, required=True, help='Trailing stop pips')
    parser.add_argument('--stop_pips', type=int, required=True, help='Stop pips for pending orders')
    parser.add_argument('--login', type=int, required=True, help='Account number')
    parser.add_argument('--password', type=str, required=True, help='Account password')
    parser.add_argument('--server', type=str, required=True, help='Broker server')

    return parser.parse_args()

# Function to convert string times to datetime objects
def convert_times(times):
    return [datetime.strptime(time_str, '%Y-%m-%d %H:%M:%S') for time_str in times]

# Parse arguments
args = parse_arguments()

# Initialize the MetaTrader 5 platform
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()
    exit()

# Login to the trading account
if not mt5.login(args.login, args.password, args.server):
    print(f"Failed to connect to account {args.login}")
    mt5.shutdown()
    exit()

# Ensure AutoTrading is enabled
account_info = mt5.account_info()
if account_info is None:
    print("Failed to get account info, AutoTrading might be disabled")
    mt5.shutdown()
    exit()
elif not account_info.trade_allowed:
    print("AutoTrading is disabled, please enable it in the MetaTrader 5 platform")
    mt5.shutdown()
    exit()

# Set parameters
symbols = args.symbols
scheduled_times = convert_times(args.times)
lot = args.lot

# Explicitly define pip values for each symbol
pip_dict = {"EURUSD": 0.0001, "GBPUSD": 0.0001, "USDJPY": 0.01, "XAUUSD": 0.1}

stop_loss_pips = args.stop_loss_pips
trigger_pips = args.trigger_pips
trailing_pips = args.trailing_pips
stop_pips = args.stop_pips

# Function to ensure the symbol is available
def ensure_symbol(symbol):
    symbol_info = mt5.symbol_info(symbol)
    if symbol_info is None:
        print(f"Symbol {symbol} not found, cannot call order_check()")
        return False
    elif not symbol_info.visible:
        print(f"Symbol {symbol} is not visible, trying to switch on")
        if not mt5.symbol_select(symbol, True):
            print(f"Symbol {symbol} is not available for trading")
            return False
    return True

# Function to place the orders
def place_orders(symbol):
    if not ensure_symbol(symbol):
        return

    pip = pip_dict[symbol]
    symbol_tick = mt5.symbol_info_tick(symbol)
    ask_price = symbol_tick.ask
    bid_price = symbol_tick.bid

    stop_loss_price_buy = ask_price - stop_loss_pips * pip
    stop_price_buy = ask_price + stop_pips * pip

    stop_loss_price_sell = bid_price + stop_loss_pips * pip
    stop_price_sell = bid_price - stop_pips * pip

    buy_stop_request = {
        "action": mt5.TRADE_ACTION_PENDING,
        "symbol": symbol,
        "volume": lot,
        "type": mt5.ORDER_TYPE_BUY_STOP,
        "price": stop_price_buy,
        "sl": stop_loss_price_buy,
        "deviation": 10,
        "magic": 234000,
        "comment": "python script buy stop",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_FOK,  # Using FOK filling mode
    }

    sell_stop_request = {
        "action": mt5.TRADE_ACTION_PENDING,
        "symbol": symbol,
        "volume": lot,
        "type": mt5.ORDER_TYPE_SELL_STOP,
        "price": stop_price_sell,
        "sl": stop_loss_price_sell,
        "deviation": 10,
        "magic": 234000,
        "comment": "python script sell stop",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_FOK,  # Using FOK filling mode
    }

    result_buy_stop = mt5.order_send(buy_stop_request)
    if result_buy_stop.retcode != mt5.TRADE_RETCODE_DONE:
        print(f"Buy stop order for {symbol} failed, retcode={result_buy_stop.retcode}")
        print(f"Error: {result_buy_stop}")
    else:
        print(f"Buy stop order for {symbol} placed successfully, ticket={result_buy_stop.order}")

    result_sell_stop = mt5.order_send(sell_stop_request)
    if result_sell_stop.retcode != mt5.TRADE_RETCODE_DONE:
        print(f"Sell stop order for {symbol} failed, retcode={result_sell_stop.retcode}")
        print(f"Error: {result_sell_stop}")
    else:
        print(f"Sell stop order for {symbol} placed successfully, ticket={result_sell_stop.order}")

# Function to close pending orders not executed within 3 minutes
def close_pending_orders(symbol, after_time):
    close_time = after_time + timedelta(minutes=3)
    while datetime.now() < close_time:
        time.sleep(1)

    orders = mt5.orders_get(symbol=symbol)
    if orders is None:
        print(f"No orders to close for {symbol}.")
        return

    for order in orders:
        if order.type in [mt5.ORDER_TYPE_BUY_STOP, mt5.ORDER_TYPE_SELL_STOP]:
            close_request = {
                "action": mt5.TRADE_ACTION_REMOVE,
                "order": order.ticket,
                "symbol": order.symbol,
                "volume": order.volume_current,
                "magic": order.magic,
                "comment": "python script cancel pending",
            }
            result = mt5.order_send(close_request)
            if result.retcode != mt5.TRADE_RETCODE_DONE:
                print(f"Failed to close order {order.ticket} for {symbol}, retcode={result.retcode}")
            else:
                print(f"Order {order.ticket} for {symbol} closed successfully.")

# Function to display the countdown timer
def display_countdown(target_time):
    while datetime.now() < target_time:
        now = datetime.now()
        remaining_time = target_time - now
        hours, remainder = divmod(remaining_time.seconds, 3600)
        minutes, seconds = divmod(remainder, 60)
        print(f"Time remaining: {remaining_time.days} days, {hours:02}:{minutes:02}:{seconds:02}", end="\r")
        time.sleep(1)

# Function to monitor and adjust trailing stops
def monitor_trailing_stops(symbol, target_time):
    pip = pip_dict[symbol]
    trigger_price_buy = None
    trigger_price_sell = None

    while datetime.now() < target_time + timedelta(minutes=30):
        positions = mt5.positions_get(symbol=symbol)
        if positions:
            for pos in positions:
                if pos.type == mt5.ORDER_TYPE_BUY:
                    if trigger_price_buy is None:
                        trigger_price_buy = pos.price_open + trigger_pips * pip
                    if mt5.symbol_info_tick(symbol).ask > trigger_price_buy:
                        new_sl = mt5.symbol_info_tick(symbol).ask - trailing_pips * pip
                        if new_sl > pos.sl:
                            request = {
                                "action": mt5.TRADE_ACTION_SLTP,
                                "position": pos.ticket,
                                "sl": new_sl,
                                "tp": pos.tp
                            }
                            result = mt5.order_send(request)
                            if result.retcode != mt5.TRADE_RETCODE_DONE:
                                print(f"Failed to modify SL for {symbol}, retcode={result.retcode}")
                            else:
                                print(f"Modified SL for {symbol} position {pos.ticket}")

                elif pos.type == mt5.ORDER_TYPE_SELL:
                    if trigger_price_sell is None:
                        trigger_price_sell = pos.price_open - trigger_pips * pip
                    if mt5.symbol_info_tick(symbol).bid < trigger_price_sell:
                        new_sl = mt5.symbol_info_tick(symbol).bid + trailing_pips * pip
                        if new_sl < pos.sl:
                            request = {
                                "action": mt5.TRADE_ACTION_SLTP,
                                "position": pos.ticket,
                                "sl": new_sl,
                                "tp": pos.tp
                            }
                            result = mt5.order_send(request)
                            if result.retcode != mt5.TRADE_RETCODE_DONE:
                                print(f"Failed to modify SL for {symbol}, retcode={result.retcode}")
                            else:
                                print(f"Modified SL for {symbol} position {pos.ticket}")

        time.sleep(1)

# Function to manage multiple timers for multiple symbols
def manage_timers(symbols, scheduled_times):
    for scheduled_time in scheduled_times:
        # Display the countdown timer
        display_countdown(scheduled_time)

        # Place the orders for each symbol
        for symbol in symbols:
            place_orders(symbol)

        # Close pending orders not taken after 3 minutes for each symbol
        for symbol in symbols:
            close_pending_orders(symbol, scheduled_time)

        # Monitor trailing stops for each symbol
        for symbol in symbols:
            monitor_trailing_stops(symbol, scheduled_time)

# Manage multiple timers for multiple symbols
manage_timers(symbols, scheduled_times)

# Shutdown the MetaTrader 5 platform
mt5.shutdown()


In [ ]:
### python script_name.py --symbols EURUSD GBPUSD USDJPY XAUUSD --times "2024-06-18 13:28:00" "2024-06-18 13:39:00" "2024-06-18 13:45:00" --lot 1.0 --stop_loss_pips 10 --trigger_pips 20 --trailing_pips 15 --stop_pips 10 --login 48438229 --password "Mq3p8!tJ" --server "HFMarketsGlobal-Demo"
